In [2]:
import io
import pandas as pd
from connections import AWS
from biomech.processing import write_mot_file
from biomech.algorithms import butter_lowpass_filter

$\textbf{Inverse Kinematics: Filtering}$

- Applies fourth order, zero-lag, 18 Hz Butterworth filter to IK results files
- Results re-saved as `.mot` files for OpenSim compatibility

In [3]:
# load a .mot file (e.g., results from an IK run)
def load_mot_file(path: str) -> pd.DataFrame:
    return pd.read_csv(path, delim_whitespace=True, skiprows=10)

In [4]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


In [13]:
# load all IK results files
s3_objects = aws_connection.list_s3_objects(prefix='biomechanics/subjects/')
mot_files = [obj for obj in s3_objects if obj.endswith('.mot') and '_ik' in obj]

In [15]:
# iterate through all files
for f in mot_files:
    # load & convert to df
    ik_bytes = aws_connection.load_s3_object(f)
    ik_df = load_mot_file(io.BytesIO(ik_bytes))

    # match angle columns based on throwing hand
    if 'arm_flex_r' in ik_df.columns:
        angle_cols = [
            'arm_flex_r', 'arm_add_r', 'arm_rot_r', 
            'elbow_flex_r', 'pro_sup_r', 'wrist_flex_r', 'wrist_dev_r'
        ]
    else:
        angle_cols = [
            'arm_flex_l', 'arm_add_l', 'arm_rot_l', 
            'elbow_flex_l', 'pro_sup_l', 'wrist_flex_l', 'wrist_dev_l'
        ]

    # apply filter
    ik_df_filt_18 = ik_df.copy()
    ik_df_filt_18 = butter_lowpass_filter(
        ik_df_filt_18,
        columns=angle_cols,
        cutoff=18,
        order=4
    )

    # write to (local) .mot file
    local_mot_path = 'trial_filtered.mot'
    write_mot_file(
        ik_df_filt_18,
        local_mot_path
    )

    # upload to S3
    subject_id = f.split('/')[-1].split('_ik')[0].split('_')[0]
    trial_id = f.split('/')[-1].split('_ik')[0]
    aws_connection.upload_to_s3(
        local_mot_path,
        f'biomechanics/subjects/{subject_id}/mot_processed/{trial_id}_ik_filtered.mot'
    )

[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_01_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_02_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_03_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_04_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_05_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_06_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_07_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_08_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/2609/mot_processed/2609_09_ik_filtered.mot
[AWS]: Uploaded object to s3://pitch-ml/biomechanics/su

$\textbf{Close Connection}$

In [16]:
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
